In [11]:
import os
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from dateutil.parser import parse
from itertools import chain
import operator
import sys
from fbprophet import Prophet
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py

from fbprophet import Prophet
from pandas.tseries.offsets import *
py.init_notebook_mode(connected=True)
%matplotlib inline



In [12]:

import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [13]:
df_result = pd.read_csv("output/df_result_fb.csv")

In [14]:
#run it against a small population
#df_result = df_result[(df_result.Stock == 830)|(df_result.Stock == 828)|(df_result.Stock == 363)|(df_result.Stock == 1223)|(df_result.Stock == 1372)]
#df_result = df_result[(df_result.Stock == 363)]
#df_result = df_result[(df_result.Stock == 1797)]
df_result.rename(columns={'Unnamed: 0': 'Pop', 'Unnamed: 1': 'Count'}, inplace=True)
df_result['ds']=pd.to_datetime(df_result['FB_Date'], format='%Y-%m-%d')
df_result.head()

,Pop,Count,Day,Index,Market,Stock,Weight,x0,x1,x2,...,x3B,x3C,x3D,x3E,x4,x5,x6,y,FB_Date,ds
0,test,0,13,0,1,363,NaN,0.000842,0.436594,0.477848,...,4.353950e-06,7.718616e-06,1.632886e-05,3.267851e-05,0.001661,9.724611e-07,112.968475,NaN,2015-01-21,2015-01-21
1,test,1,13,1,1,1223,NaN,0.020800,0.636143,0.670798,...,9.546012e-06,1.575258e-05,2.804666e-05,1.082500e-05,0.002104,2.146299e-05,100.000000,NaN,2015-01-21,2015-01-21
2,test,2,13,2,1,1372,NaN,0.068580,0.478891,0.481816,...,1.733539e-07,3.866963e-07,8.520664e-07,2.103176e-06,0.000222,3.085296e-08,188.335508,NaN,2015-01-21,2015-01-21
3,test,3,13,3,1,1942,NaN,0.011986,0.463201,0.294900,...,3.555558e-08,7.280376e-08,1.364231e-07,3.155050e-07,0.000141,1.678216e-08,212.123913,NaN,2015-01-21,2015-01-21
4,test,4,13,4,1,2712,NaN,0.000164,0.549980,0.352332,...,8.503405e-07,1.222847e-06,1.901114e-06,4.048304e-06,0.001235,4.595200e-07,124.024977,NaN,2015-01-21,2015-01-21


In [15]:
def run_fb(row):
    Market = row['Market']
    Stock = row['Stock']
    period = row['count']
    #Step 1 get the dataframe for the Market and Security from df
    df_train=df_result[(df_result.Stock == Stock) & (df_result.Market == Market)]
    try:
        df_train = df_train.loc[:,['FB_Date','y']]
        df_train['ds'] = df_train['FB_Date']
        df_train.set_index('FB_Date',inplace=True)
        df_train = df_train.loc[:,['ds','y']]
        df_train.sort_index(axis=0,ascending=True,inplace=True)
        df_train = df_train.reset_index(drop=True)
        m = Prophet()
        m.fit(df_train)
        future = m.make_future_dataframe(periods=period)
        forecast = m.predict(future)
        #print(forecast['yhat'])
        df = pd.DataFrame({'Market': Market, 'Stock': Stock, 'ds': forecast['ds'], 'yhat': forecast['yhat'] })
        return ( df )
    except: 
        print("Came to except")
        df = pd.DataFrame({'Market': Market, 'Stock': Stock, 'ds': df_train['ds'], 'yhat': 0.0 } )
        return (df)

In [16]:
#s.groupby(level=['first', 'second']).sum()
grouped = pd.DataFrame({'count' : df_result.groupby(['Market','Stock']).size()} ).reset_index()

In [17]:
grouped.head()

,Market,Stock,count
0,1,10,502
1,1,12,502
2,1,15,502
3,1,20,502
4,1,22,300


In [18]:
value = grouped['Stock'].count()

In [19]:
print( str(value))

3023


In [20]:
#grouped = df_result.groupby(['Market','Stock'])['Day'].count()

In [ ]:
df_output = pd.DataFrame()
i=0
for index, row in grouped.iterrows():
    i=i+1
    print("Market and Stock:"+ str(row['Market'])+"Stock:"+str(row['Stock'])+" For"+str(i) +" OF:"+str(value) )
    try:
        df_output=pd.concat([df_output,run_fb(row)])
    except:
        continue
        #df_result.merge( df_output, left_on=['Market','Stock','ds'],right_on=['Market','Stock','ds'], how='inner', indicator='_merge')

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Market and Stock:1Stock:10 For1 OF:3023


C:\Users\kundami\Anaconda3\envs\PythonData\lib\site-packages\pystan\misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Market and Stock:1Stock:12 For2 OF:3023


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Market and Stock:1Stock:2879 For437 OF:3023


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Market and Stock:1Stock:2882 For438 OF:3023


INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Market and Stock:1Stock:2887 For439 OF:3023


In [53]:
df_output

,Market,Stock,ds,yhat
0,1,363,2015-01-05,0.000218
1,1,363,2015-01-06,0.000305
2,1,363,2015-01-09,-0.000168
3,1,363,2015-01-12,0.000229
4,1,363,2015-01-13,0.000316
5,1,363,2015-01-14,0.000356
6,1,363,2015-01-15,0.000174
7,1,363,2015-01-20,0.000326
8,1,363,2015-01-21,0.000366
9,1,363,2015-01-22,0.000184


In [ ]:
df_merge=pd.merge(left=df_result, right=df_output, left_on=['Market','Stock','ds'],right_on=['Market','Stock','ds'], how='left')

In [ ]:
df_sub = df_merge[df_merge.Pop=='test']

In [ ]:
df_sub[df_sub.Stock == 363]

In [ ]:
df_sub.to_csv("output/submission.csv")